In [ ]:
#Our project is two steps:
#1. Detect if a given package has suspicious activity (a peak or a step in downloads)
#2. Determine whether that suspicious activity can be explained.

#Step 1 is doing the "magic math" and stuff on our pkg_list. (Already completed, but needs further validation)

#Step 2 has many parts. 
#2.1 Was the package added or dropped as a dependency on a popular project around the time of the rise/ fall of downloads? (Next step)
#2.2 Did any dependents have a similar rise/ fall in downloads? (In progress, but is proving very unreliable so may be dropped). 
#2.2 is probably as far as we will be able to make it, but we can discuss further steps in our presentation
#2.3 Did any developer/ nightly builds have our package as a dependency? 
#2.4 Did a new version release that correlates to increased downloads?
#Etc

In [1]:
import gzip, json
with gzip.open('/data/NPMvulnerabilities/NPMPackages.json.gz', 'rb') as f:
    fc = f.read().decode("utf-8")
from json import JSONDecoder, JSONDecodeError
import re

NOT_WHITESPACE = re.compile(r'[^\s]')

def decode_stacked(document, pos=0, decoder=JSONDecoder()):
    while True:
        match = NOT_WHITESPACE.search(document, pos)
        if not match:
            return
        pos = match.start()

        try:
            obj, pos = decoder.raw_decode(document, pos)
        except JSONDecodeError:
            # do something sensible if there's some error
            print("DECODE_STACKED() FAILED")
            pass
        yield obj

In [ ]:
for k in dependent.keys():
    if k == 'ajv-keywords':
        print(k, dependent[k][0])
        break

In [2]:
#pkgs = ['htmlnano', 'object.map', 'babylon-walk', 'wix-style-react', 'threads', 'minimist-options', 'lead', '@babel/polyfill', '@babel/helper-define-map', 'cloudscraper']
pkgs = ['ajv-keywords', 'htmlnano']   
dependent = {}
for pkg in pkgs:
    dependent[pkg] = []
    for obj in decode_stacked(fc):
        ind = 0
        try: versions = list(obj['versions'].keys())
        except: continue
        try: release_dates = obj['time']
        except: continue
        info = {key: {'release_date': release_dates[key]} for key in release_dates if key not in ['created', 'modified']}
        for key in versions:
            try:
                info[key]['dependencies'] = list(obj['versions'][key]['dependencies'].keys())
                if pkg in info[key]['dependencies']: ind = 1
            except: continue
        if ind :
            dependent[pkg].append(obj['name'])

In [4]:
for k in dependent.keys():
    print(k)
    print(dependent[k])
    print("len = ", len(dependent[k]))
    #break

ajv-keywords
['jsonscript-proxy', 'senasaikou-yeoman', 'webpack-egoist', 'alex-d.js', 'advanced-image-loader', 'webpack-addons-ylvis', 'primeng-custom', 'jicli', '@lgeiger/electron-builder', '@touched/map-tool-project', 'generator-arc', 'node-private-tools', 'fhir2', 'webpack-ie8', 'mocoolka-tools', 'misstep', '@cysonius/schema-utils', 'ng2-dnd-kokatsuna', 'qweb-client', '@johnf/electron-builder', 'qor', 'json-framework', 'hy-checkbox', 'babel-plugin-react-native-css', 'webpack-universal', 'babel-plugin-react-css-modules-avion', 'general-req', '@ngxvoice/ngx-voicelistner', 'electron-builder-admin', 'iqm', 'freemamba', 'isomorphic-webpack', 'webpack-tal', 'search-list-react', 'sea-floor', 'carousel-react', 'nicholas_util', 'search-input-react', 'json-injector', 'es-webpack', '@frctl/support', 'ajv-interface-keywords', 'stygian', 'lrbceshi', 'gh-lint', '@jose_santacruz/middy', '@ddder/webpack', 'vbot', 'outils-ren', 'canvas-fingerprint', 'firebase-forum', 'firebase-feedback', 'egeria-mer

In [17]:
import time, requests, sys, pymongo, json, datetime


dbname = "NPMvulnerabilities"
collname = "pkg_dl2"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]

pkgs = ['ajv-keywords', 'htmlnano']
#pkgs = ['htmlnano']
start_date = datetime.datetime(2015, 3, 5)
delta = datetime.timedelta(days=6)
end_date = start_date + delta
dt_range = []
final_date = datetime.datetime(2018, 10, 3)

while start_date < final_date:
    if end_date > final_date : end_date = final_date
    dt_range.append((start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))
    start_date = end_date + datetime.timedelta(days=1)
    end_date = start_date + delta


# get data
base_url = 'https://api.npmjs.org/downloads/point/'
l = len(pkgs)
print(l)
for pkg in pkgs:
    dl = []
    for r in dt_range:
        url = base_url+r[0]+':'+r[1]+'/'+pkg
        try: r = requests.get(url)
        except: print('Timeout '+ pkg+'\n')
        
        try:
            result = r.content
            try:
                rj = json.loads(result.decode('utf-8', errors='ignore'))
                rj.pop('package', None)
                dl = dl + [rj]
            except: print('DecodeError '+ pkg+'\n')
        except: print('BadURL '+ pkg+'\n')
    #ent = {pkg: dl}    
    ent = {'name':pkg, 'dl': dl}

    coll.insert_one(ent)
    print(ent)

2
{'dl': [{'start': '2015-03-05', 'downloads': 0, 'end': '2015-03-11'}, {'start': '2015-03-12', 'downloads': 0, 'end': '2015-03-18'}, {'start': '2015-03-19', 'downloads': 0, 'end': '2015-03-25'}, {'start': '2015-03-26', 'downloads': 0, 'end': '2015-04-01'}, {'start': '2015-04-02', 'downloads': 0, 'end': '2015-04-08'}, {'start': '2015-04-09', 'downloads': 0, 'end': '2015-04-15'}, {'start': '2015-04-16', 'downloads': 0, 'end': '2015-04-22'}, {'start': '2015-04-23', 'downloads': 0, 'end': '2015-04-29'}, {'start': '2015-04-30', 'downloads': 0, 'end': '2015-05-06'}, {'start': '2015-05-07', 'downloads': 0, 'end': '2015-05-13'}, {'start': '2015-05-14', 'downloads': 0, 'end': '2015-05-20'}, {'start': '2015-05-21', 'downloads': 0, 'end': '2015-05-27'}, {'start': '2015-05-28', 'downloads': 0, 'end': '2015-06-03'}, {'start': '2015-06-04', 'downloads': 0, 'end': '2015-06-10'}, {'start': '2015-06-11', 'downloads': 0, 'end': '2015-06-17'}, {'start': '2015-06-18', 'downloads': 0, 'end': '2015-06-24'}

{'dl': [{'start': '2015-03-05', 'downloads': 0, 'end': '2015-03-11'}, {'start': '2015-03-12', 'downloads': 0, 'end': '2015-03-18'}, {'start': '2015-03-19', 'downloads': 0, 'end': '2015-03-25'}, {'start': '2015-03-26', 'downloads': 0, 'end': '2015-04-01'}, {'start': '2015-04-02', 'downloads': 0, 'end': '2015-04-08'}, {'start': '2015-04-09', 'downloads': 0, 'end': '2015-04-15'}, {'start': '2015-04-16', 'downloads': 0, 'end': '2015-04-22'}, {'start': '2015-04-23', 'downloads': 0, 'end': '2015-04-29'}, {'start': '2015-04-30', 'downloads': 0, 'end': '2015-05-06'}, {'start': '2015-05-07', 'downloads': 0, 'end': '2015-05-13'}, {'start': '2015-05-14', 'downloads': 0, 'end': '2015-05-20'}, {'start': '2015-05-21', 'downloads': 0, 'end': '2015-05-27'}, {'start': '2015-05-28', 'downloads': 0, 'end': '2015-06-03'}, {'start': '2015-06-04', 'downloads': 0, 'end': '2015-06-10'}, {'start': '2015-06-11', 'downloads': 0, 'end': '2015-06-17'}, {'start': '2015-06-18', 'downloads': 0, 'end': '2015-06-24'}, 

In [ ]:
import math
total_list = []
start_date = []
end_date = []
download_list = []

for r in coll.find():
    download_list = []
    start_date = []
    ent = r['dl']
    for k in ent:
        download_list.append(k['downloads'])
        start_date.append(k["start"])
#map, reduce, filter, lambda
    log_list = [math.log(x+1) for x in download_list]
    difference_list = []
    for i in range(1,len(log_list)-1):
        dif = log_list[i] - log_list[i-1]
        difference_list.append( (log_list[i] - log_list[i-1]))
        if dif > 7:
            print("Start date of spike for " + r['name'] +": " +start_date[i])
        if dif < -7:
            print("Start date of fall for " + r['name'] +": " +start_date[i])
    #print(difference_list)
#print(log_list)
#print(download_list)


In [69]:
import time, requests, sys, pymongo, json, datetime


dbname = "NPMvulnerabilities"
collname = "dep_dl"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]

#pkgs = ['ajv-keywords', 'htmlnano']
start_date = datetime.datetime(2015, 3, 1)
delta = datetime.timedelta(days=6)
end_date = start_date + delta
dt_range = []
final_date = datetime.datetime(2018, 10, 1)

while start_date < final_date:
    if end_date > final_date : end_date = final_date
    dt_range.append((start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))
    start_date = end_date + datetime.timedelta(days=1)
    end_date = start_date + delta


# get data
base_url = 'https://api.npmjs.org/downloads/point/'
l = len(pkgs)
print(l)
for k in dependent.keys():
    for i in dependent[k]:
        dl = []
        for r in dt_range:
            url = base_url+r[0]+':'+r[1]+'/'+i
            try: r = requests.get(url)
            except: print('Timeout '+ i+'\n')
        
            try:
                result = r.content
                try:
                    rj = json.loads(result.decode('utf-8', errors='ignore'))
                    rj.pop('package', None)
                    dl = dl + [rj]
                except: print('DecodeError '+ i+'\n')
            except: print('BadURL '+ i+'\n')
        #ent = {pkg: dl}    
        ent = {'name':i, 'dl': dl}
        coll.insert_one(ent)
        print(ent)

1


TypeError: Can't convert 'set' object to str implicitly

In [80]:
import time, requests, sys, pymongo, json, datetime


dbname = "NPMvulnerabilities"
collname = "dep_dl"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]

for r in coll.find():
    temp = r['name']
    print(temp)

jsonscript-proxy
senasaikou-yeoman
webpack-egoist
alex-d.js
advanced-image-loader
webpack-addons-ylvis
primeng-custom
jicli
@lgeiger/electron-builder
@touched/map-tool-project
generator-arc
node-private-tools
fhir2
webpack-ie8
mocoolka-tools
misstep
@cysonius/schema-utils
ng2-dnd-kokatsuna
qweb-client
@johnf/electron-builder
qor
json-framework
hy-checkbox
babel-plugin-react-native-css
webpack-universal
babel-plugin-react-css-modules-avion
general-req
@ngxvoice/ngx-voicelistner
electron-builder-admin
iqm
freemamba
isomorphic-webpack
webpack-tal
search-list-react
sea-floor
carousel-react
nicholas_util
search-input-react
json-injector
es-webpack
@frctl/support
ajv-interface-keywords
stygian
lrbceshi
gh-lint
@jose_santacruz/middy
@ddder/webpack
vbot
outils-ren
canvas-fingerprint
firebase-forum
firebase-feedback
egeria-mercurius
webpack-no-postintall-fail
webpack-no-postinstall-fail
calendar-source-provider
egg-ajv-keywords
eslint-plugin-aurelia-event-aggregator
appache
comanche
webpack_yb


In [18]:
import math, time, requests, sys, pymongo, json, datetime

dbname = "NPMvulnerabilities"
collname = "pkg_dl"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]
for r in coll.find():
    print("for package " + r['name'] + ":")
    download_list = []
    start_date = []
    ent = r['dl']
    for k in ent:
        download_list.append(k["downloads"])
        start_date.append(k["start"])
            
    for x in range(1,len(download_list) -1):
        if download_list[x] > 100:
            if (download_list[x] > 0) and (download_list[x-1] > 0):
                pdif = (download_list[x] - download_list[x-1]) / download_list[x-1]
                if .3 <= pdif:
                    print("hey we have a rise for " + r['name'] + " at " + start_date[x])
                if -.3 >= pdif:
                    print("hey we have a fall for " + r['name'] + " at " + start_date[x])


for package ajv-keywords:
hey we have a rise for ajv-keywords at 2016-09-15
hey we have a rise for ajv-keywords at 2016-09-22
hey we have a rise for ajv-keywords at 2017-01-05
hey we have a rise for ajv-keywords at 2018-01-04
hey we have a fall for ajv-keywords at 2018-05-24
hey we have a rise for ajv-keywords at 2018-06-07
for package htmlnano:
hey we have a rise for htmlnano at 2017-11-30
hey we have a rise for htmlnano at 2017-12-07
hey we have a rise for htmlnano at 2017-12-21
hey we have a rise for htmlnano at 2017-12-28
hey we have a fall for htmlnano at 2018-01-04
hey we have a fall for htmlnano at 2018-05-24
hey we have a rise for htmlnano at 2018-06-07
hey we have a rise for htmlnano at 2018-08-30


In [13]:
#revert to previous version and gets the old format of the versions. Gets dependent list with version history
#pkgs = ['ajv-keywords']   
dependent1 = {}
for pkg in pkgs:
    dependent1[pkg] = []   
    for obj in decode_stacked(fc):
        ind = 0
        try: versions = list(obj['versions'].keys())
        except: continue
        try: release_dates = obj['time']
        except: continue
        info = {key: {'release_date': release_dates[key]} for key in release_dates if key not in ['created', 'modified']}
        for key in versions:
            try:
                info[key]['dependencies'] = list(obj['versions'][key]['dependencies'].keys())
                if pkg in info[key]['dependencies']: ind = 1
            except: continue
        if ind :
            dependent1[pkg].append({obj['name']: info})

In [19]:
for k in dependent1.keys():
    print(dependent1[k][1:2])
    print("len = ", len(dependent1[k]))
    break

[{'tzar-core': {'2.3.1': {'dependencies': ['elasticlunr', 'chai-jquery', 'jquery', 'richtypo', 'postcss-cssnext', 'JSONStream', 'express', 'chalk', 'pdf-merge', 'dddos', 'postcss-loader', 'posthtml-inline-css', 'localforage', 'gulp-babel', 'js-yaml', 'babel-loader', 'pad-number', 'gulp-nodemon', 'zebra-typo', 'postcss-filter-gradient', 'memoizerific', 'transliteration.cyr', 'jquery-validation', 'moment', 'html-pdf', 'mocha', 'gulp-graph', 'css-loader', 'gulp-svgmin', 'phantom', 'chai-files', 'babel-plugin-transform-es3-property-literals', 'babel-cli', 'postcss-opacity', 'raw-loader', 'postcss-discard-duplicates', 'apply-loader', 'file-saver', 'passport-local', 'filesize', 'cli-columns', 'gulp-util', 'stylefmt', 'cheerio', 'memory-cache', 'antimoderate', 'mkdirp', 'jquery-ui', 'modernizr', 'urijs', 'precss', 'noty', 'postcss-svgo', 'gulp-sourcemaps', 'jsdom', 'merge-stream', 'striptags', 'htmlnano', 'mdoc', 'csv-parse', 'passport', 'gulp-compass', 'gulp-spawn-mocha', 'gulp-live-server',

In [20]:
# dict - {target package : [(dependent package, release)]}
from datetime import datetime as dt
delrl = {}
adrl = {}
all_del_deps =[]
for k in dependent1.keys():
    adrl[k] = [] #list of all versions with dependency where previous version did not have it
    delrl[k] = [] #list of all version without dependency where previous version did have it
    for l in dependent1[k]:
        d = list(l.keys())[0]
        v = l[d]
        rdt_l = {}


        for rk in v.keys():
            r = v[rk]

            rdt = r['release_date'][:10]+' '+r['release_date'][11:19]
            rdt = dt.strptime(rdt , "%Y-%m-%d %H:%M:%S")
            try:
                if k in r['dependencies']: rdt_l[rdt] = ['Y',rk]
                else: rdt_l[rdt] = ['N', rk]
            except:
                continue

        rdt_s = sorted(rdt_l.keys())
        fadd = 0

        for r in rdt_s:
            if fadd == 0 and rdt_l[r][0] == 'Y':
                if r.date() < dt(2015,3,1).date(): t = dt(2015,3,1).date()
                else: t = r.date()
                adrl[k].append([d,  rdt_l[r][1], str(r)])
                fadd = 1
            if fadd and rdt_l[r][0] == 'N':
                if r.date() < dt(2015,3,1).date(): t = dt(2015,3,1).date()
                else: t = r.date()
                delrl[k].append([d, rdt_l[r][1], str(r)])
                all_del_deps.append(d)
                fadd = 0

In [21]:
for k in adrl:
    for r in range(0, len(adrl[k])):
        print(adrl[k][r])

['gulp-htmlnano', '0.1.0', '2016-01-20 16:04:15']
['tzar-core', '2.3.1', '2017-03-23 15:17:55']
['parsley-bundler', '0.0.0', '2017-12-21 09:13:47']
['cy-parcel', '1.4.1', '2018-01-18 02:02:59']
['htm-asset-combo', '1.0.0', '2018-01-23 03:31:04']
['parcel-bundler-vue', '1.6.2', '2018-02-21 09:08:57']
['kwasniew-parcel-bundler', '1.6.2', '2018-03-17 11:36:06']
['@ash9g/parcel-bundler', '1.7.0', '2018-04-01 04:39:58']
['baidu-lego-events', '0.0.2', '2018-04-03 06:33:17']
['html-loader', '1.0.0-alpha.0', '2018-02-06 03:37:54']
['@alexlafroscia/parcel-bundler', '1.9.3', '2018-06-25 23:43:56']
['parcel-bundler', '1.0.0-alpha.1', '2017-10-24 20:42:01']
['parcel', '1.8.0', '2018-05-03 18:53:47']
['@erickmerchant/optimize', '9.3.0', '2018-06-05 01:45:16']
['jgb-plugin-html', '1.2.4', '2018-08-31 03:21:12']
